To limit the use of GPT and the costs for the API only pass a reduced dataset to the LLM   
To controll this behavior we have to rename the columns

In [1]:
import pandas as pd
import numpy as np
import string
import random

In [35]:
test = pd.read_csv('../dessi-mf/test.csv')
test_labels = pd.read_csv('../dessi-mf/test_labels_pii.csv')
test_classes = pd.read_csv('../dessi-mf/test_classes.csv')
test_dataset = pd.read_csv('../dessi-mf/test_dataset.csv')

In [36]:
test_classes_dataset = test_classes.copy()
for i in range(test_classes_dataset.shape[0]):
    test_classes_dataset.iloc[i, 0] = test_classes_dataset.iloc[i, 0] + "_" + test_dataset.iloc[i, 0]
test_classes_dataset

,class
0,pyint_de_DE_faker
1,credit_card_number_fr_mimesis
2,company_en_faker
3,credit_card_provider_fr_FR_faker
4,academic_degree_en_mimesis
...,...
14733,blood_type_fr_mimesis
14734,email_en_faker
14735,title_de_mimesis
14736,Date_dessi


In [37]:
cols = []
for i in test_classes_dataset["class"].unique():
    cols.append(list(test_classes_dataset[test_classes_dataset["class"] == i].index[:2]))

In [38]:
random.seed(42)
def generate_random_string(length):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

df_llm = test.iloc[:, np.array(cols).flatten()]
col_names = []
for i in test_classes_dataset["class"].unique():
    col_names.append(i)
    random_length = random.randint(5, 20)
    random_string = generate_random_string(random_length)
    col_names.append(random_string)
    
df_llm.columns = col_names
df_llm

,pyint_de_DE_faker,bVrpoiVg,credit_card_number_fr_mimesis,LBcbfno,company_en_faker,JmTPS,credit_card_provider_fr_FR_faker,oCLrZ3aWZkSBvrjn9W,academic_degree_en_mimesis,gfygw2wMqZcUDIh,...,version_de_mimesis,YHuSAAjtylWIEp2ot,answer_de_mimesis,ZD6dJA9AJ,ID_Card_dessi,ypqnvPf7C,resolution_fr_mimesis,fIU1mdryRMMc3emZ,Email_dessi,En36vt96an7
0,9150,7025,4565 7937 2995 3231,4632 9118 7223 0178,Wu LLC,Graham-Davis,VISA 16 digit,VISA 19 digit,Master,Bachelor,...,48.14.45,56.34.54,Vielleicht,Nein,U1808324,T4886476,1920x1200,1280x1024,bethel37@hotmail.com,dstokes@hane.com
1,2566,7780,3772 946671 16846,4218 6228 8296 0710,Meadows and Sons,"Monroe, Mclaughlin and Dunlap",JCB 16 digit,VISA 16 digit,Bachelor,Bachelor,...,27.34.30,54.38.11,Nein,Nein,F2198990,P4147520,2880x1920,2560x2048,henri.nader@effertz.com,corwin.nico@gmail.com
2,3909,3331,4532 0710 1134 6723,5354 4450 7075 4302,"Harrell, White and Bowers","Figueroa, Manning and Flores",Mastercard,Discover,Master,Bachelor,...,19.21.75,89.60.31,Ja,Ja,M9197125,W7949367,1152x768,2560x2048,larson.devyn@durgan.com,joyce.heathcote@yahoo.com
3,4574,4140,3469 556889 86403,3465 699897 38890,Hernandez Inc,Smith Group,VISA 16 digit,JCB 16 digit,PhD,PhD,...,96.0.61,25.61.60,Nein,Vielleicht,D5126714,H2064969,2560x2048,2560x2048,jromaguera@kulas.net,cormier.alexandra@yahoo.com
4,8729,9297,3420 971634 58790,4906 8673 8386 0110,Wagner Ltd,"Logan, Cunningham and Brown",Diners Club / Carte Blanche,VISA 19 digit,Bachelor,PhD,...,44.43.98,34.9.94,Vielleicht,Nein,Z4823692,Z8740802,1440x960,1440x960,marcellus.heidenreich@yahoo.com,vicenta23@zboncak.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,6462,5031,4699 4639 9901 2014,3466 127091 74366,Hall PLC,Aguirre Group,Mastercard,VISA 16 digit,PhD,PhD,...,35.7.94,22.6.43,Ja,Ja,Z6329403,D0124844,1280x768,1600x1200,jacky73@hotmail.com,ubergstrom@hermiston.org
96,8141,2653,4367 6055 9557 3093,4868 2814 1132 7751,Carey-Taylor,"Parker, Vasquez and Smith",Mastercard,Maestro,Bachelor,PhD,...,26.74.23,94.41.59,Vielleicht,Nein,K2871718,T4817090,1600x900,1440x960,lenore.howell@yahoo.com,vidal84@gmail.com
97,3319,7137,2656 7037 8646 7219,4291 2962 1347 7007,Graves Inc,King-Barnett,VISA 13 digit,JCB 16 digit,Bachelor,Bachelor,...,57.31.70,72.92.98,Ja,Vielleicht,K0858749,U1940415,3200x2400,2048x1536,bruen.isidro@yahoo.com,verlie.bergnaum@yahoo.com
98,320,6801,5494 7468 2743 9563,4058 6926 0206 1498,Delacruz and Sons,Thompson-Phillips,American Express,American Express,Bachelor,Bachelor,...,30.42.72,72.41.24,Vielleicht,Vielleicht,G8187063,A8193263,1600x900,1440x900,kquitzon@walsh.com,shanie.morissette@hotmail.com


In [39]:
labels_llm = test_labels.loc[np.array(cols).flatten()].reset_index(drop=True)
classes_llm = test_classes.loc[np.array(cols).flatten()].reset_index(drop=True)
dataset_llm = test_dataset.loc[np.array(cols).flatten()].reset_index(drop=True)

In [40]:
df_llm = df_llm.sample(frac=1, axis=1, random_state=42)
labels_llm = labels_llm.sample(frac=1, axis=0, random_state=42)
classes_llm = classes_llm.sample(frac=1, axis=0, random_state=42)
dataset_llm = dataset_llm.sample(frac=1, axis=0, random_state=42)

In [41]:
df_llm.to_csv('test.csv', index=False)
labels_llm.to_csv('test_labels_pii.csv', index=False)
classes_llm.to_csv('test_classes.csv', index=False)
dataset_llm.to_csv('test_dataset.csv', index=False)

In [42]:
manual_mapping_personal = {
    "CCN": "personal",
    "Date": "non-personal",
    'Date,NIN': "personal",
    "Email": "personal",
    "Email,NIN": "personal",
    'Email,Phone_number' : "personal",
    "GPE": "non-personal",
    "Gender": "personal",
    "Geolocation": "personal",
    "IBAN": "personal",
    "ID_Card": "personal",
    "NIN": "personal",
    "NIN,Date": "personal",
    "NIN,Email" : "personal",
    "NIN,Phone_number" : "personal",
    "Nationality": "personal",
    "Organization": "non-personal",
    "Passport": "personal",
    "Phone_number": "personal",
    "Phone_number,Email": "personal",
    "Phone_number,NIN" : "personal",
    "Race": "personal",
    "Religion": "personal",
    "SWIFT/BIC": "non-personal",
    "Sexuality": "personal",
    "academic_degree": "personal",
    "address": "personal",
    "answer": "non-personal",
    "blood_type": "personal",
    "blood_group": "personal",
    "city": "non-personal",
    "color": "non-personal",
    "company": "non-personal",
    "cpu": "non-personal",
    "credit_card_number": "personal",
    "credit_card_provider": "non-personal",
    "currency": "non-personal",
    "current_location": "personal",
    "date": "non-personal",
    'dish': "non-personal",
    'drink': "non-personal",
    'duration': "non-personal",
    'ean': "non-personal",
    'email': "personal",
    'first_name': "personal",
    'float_number': "non-personal",
    'full_name': "personal",
    'gender': "personal",
    'graphics': "non-personal",
    'iban': "personal",
    'integer_number': "non-personal",
    'isbn': "non-personal",
    'isbn13': "non-personal",
    'job': "personal",
    'language': "personal",
    'last_name': "personal",
    'manufacturer': "non-personal",
    'measure_unit': "non-personal",
    'name': "personal",
    'nationality': "personal",
    'occupation': "personal",
    'passport_number': "personal",
    'phone_model': "non-personal",
    'phone_number': "personal",
    'political_views': "personal",
    'programming_language': "non-personal",
    'pyfloat': "non-personal",
    'pyint': "non-personal",
    'resolution': "non-personal",
    'sex': "personal",
    'ssn': "personal",
    'swift': "non-personal",
    'system_quality_attribute': "non-personal",
    'title': "personal",
    'url': "non-personal",
    'user_agent': "non-personal",
    'version': "non-personal",
    'word': "non-personal",
    'worldview': "personal"
}

mapping_multiclass = {
    "ccn": "credit_card_number",
    "current_location": "longitude_and_latitude",
    "geolocation": "longitude_and_latitude",
    "name": "full_name",
    "isbn13": "isbn",
    "nin": "national_identification_number",   #rename some classes so that GPT can understand them
    "ssn": "national_identification_number",
    "pyfloat": "float_number",
    "pyint": "integer_number",
    "swift": "SWIFT/BIC code",
    "swift/bic": "SWIFT/BIC code",
    "address": "full_address",
    "ean": "EAN_code",
    "occupation": "job",
    "organization": "company",
    "organization,phone_number": "company,phone_number",
    "passport": "passport_number",
    "religion": "religion/worldview",   #values in these columns contain both classes
    "worldview": "religion/worldview",
    "academic_degree": "academic_degree/title",          
    "title": "academic_degree/title",
    "blood_type": "blood_group",
    "sex": "gender"
}

In [43]:
labels_personal = classes_llm.copy()
for i in range(len(labels_personal)):
    if "mixed" in labels_personal.iloc[i,0] or "de_DE" in labels_personal.iloc[i,0] or "fr_FR" in labels_personal.iloc[i,0]:
        cla = labels_personal.iloc[i,0][:-6]
    elif "_en" in labels_personal.iloc[i,0] or "_de" in labels_personal.iloc[i,0] or "_fr" in labels_personal.iloc[i,0]:
        cla = labels_personal.iloc[i,0][:-3]
    else:
        cla = labels_personal.iloc[i,0]
    labels_personal.iloc[i,0] = manual_mapping_personal[cla]
labels_personal.rename(columns={"class": "label"}).to_csv("test_labels_personal.csv", index=False)

In [44]:
multiclass_llm = classes_llm.copy()
for i in range(multiclass_llm.shape[0]):
    if "_fr_FR" in multiclass_llm.iloc[i, 0] or "_de_DE" in multiclass_llm.iloc[i, 0] or "mixed" in multiclass_llm.iloc[i, 0]:
        multiclass_llm.iloc[i, 0] = str(multiclass_llm.iloc[i, 0])[:-6].lower()
    elif "_en" in multiclass_llm.iloc[i, 0] or "_de" in multiclass_llm.iloc[i, 0] or "_fr" in multiclass_llm.iloc[i, 0]:
        multiclass_llm.iloc[i, 0] = str(multiclass_llm.iloc[i, 0])[:-3].lower()
    else:
        multiclass_llm.iloc[i, 0] = str(multiclass_llm.iloc[i, 0]).lower()

for i in range(multiclass_llm.shape[0]):
    for a in mapping_multiclass.keys():
        if (a == multiclass_llm.iloc[i,0]) or (a in multiclass_llm.iloc[i, 0] and "," in multiclass_llm.iloc[i, 0]):
            multiclass_llm.iloc[i, 0] = multiclass_llm.iloc[i, 0].replace(a, mapping_multiclass[a])
multiclass_llm.rename(columns={"class": "label"}).to_csv("test_labels_multiclass.csv", index=False)